In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import hist
from hist import Hist

import time

import seaborn as sns
import pandas as pd

import os

from analysis_variables import *  
import babar_analysis_tools as bat


In [ ]:
BNC_tag = ""
BNC_bool = False

# BNC
#BNC_tag = "_BNC"
#BNC_bool = True

#####################################################################
# Where are we running this?
#####################################################################
## Bellis computer
topdir= "/home/bellis/babar_data_local/bnv_plambda"

if BNC_bool:
    topdir= "/home/bellis/babar_data/bnv_plambda_bnc"

## My laptop
#topdir= "/Users/josieswann/BaBar_analyses/BNV_pLambda/"
#####################################################################


#####################################################################
# Get the BNV data
#####################################################################
#data, data_collision = bat.load_datasets(topdir=topdir, subset='Run1')
#data, data_collision = bat.load_datasets(topdir=topdir, subset='all')


#####################################################################
# Get the BNC data
#####################################################################
#topdir= "/home/bellis/babar_data/bnv_plambda_bnc"
data, data_collision = bat.load_datasets(topdir=topdir, BNC=BNC_bool, subset='all')
#data, data_collision = bat.load_datasets(topdir=topdir, BNC=True, subset='Run1')
#BNC_tag = "_BNC"
#BNC_bool = True

#####################################################################
# Get the BNC data
#####################################################################
#topdir= "/home/bellis/babar_data/bnv_plambda_bnc"
#data, data_collision = bat.load_datasets(topdir=topdir, BNC=True, subset='all')
#data, data_collision = bat.load_datasets(topdir=topdir, BNC=True, subset='Run1')



In [ ]:
from analysis_variables import *

region_definitions

In [ ]:
data["BpostFitDeltaE"]

In [ ]:
# SP
# Need to get the original duplicates mask for any other cuts we might generate outside the function
dcuts = bat.get_final_masks(data, region_definitions=region_definitions)

print([dcuts.keys()])
print()

for key in dcuts.keys():
    print(f'{key:3d} {dcuts[key]["name"]}')

print()

# Collision data
# Need to get the original duplicates mask for any other cuts we might generate outside the function
dcuts_coll = bat.get_final_masks(data_collision, region_definitions=region_definitions)

print([dcuts_coll.keys()])
print()

for key in dcuts_coll.keys():
    print(f'{key:3d} {dcuts_coll[key]["name"]}')



In [ ]:
# BNC
mask = dcuts[1]['event'] & dcuts[2]['event'] & dcuts[3]['event'] & dcuts[4]['event'] 
dcuts[-2] = {'name':'all - BNC', 'event':mask, 'candidates':None} 

mask = dcuts_coll[1]['event'] & dcuts_coll[2]['event'] & dcuts_coll[3]['event'] & dcuts_coll[4]['event'] 
dcuts_coll[-2] = {'name':'all - BNC', 'event':mask, 'candidates':None} 

In [ ]:
dataset_information = pd.read_csv("dataset_statistics.csv")
cs_data= pd.read_csv("SP_cross_sections_and_labels.csv")

no_notes= cs_data.drop(["Uncertainty","Note: cross sections found at https://babar-wiki.heprc.uvic.ca/bbr_wiki/index.php/Physics/Cross_sections,_luminosities,_and_other_vital_stats"], axis= 1)
no_notes



In [ ]:
bkg_spmodes = ['998', '1005', '1235', '1237', '3981']
sig_spmodes = ['-999']

spmodes = bkg_spmodes + sig_spmodes

weights = {}
for sp in spmodes:
    weights[sp] = bat.scaling_value(int(sp), dataset_information=dataset_information, cs_data=cs_data, plot=False, verbose=False)
    #weights[sp] = 1

### bat.scaling_value is in Babar_analysis_tools.py 

# Scale the signal higher
weights['-999'] = 1000
weights['0'] = 1

print(weights)

In [ ]:
all_hists= bat.create_empty_histograms(hist_defs)


In [ ]:
x = list(all_hists.keys())
print(x)

In [ ]:

'''
subset = ['BSphr', 'BpostFitDeltaE', 'BpostFitMes', 'BThrust', 'BCosThetaS', \
         'Lambda0_unc_Mass', 'R2', 'R2All', \
          'thrustMag', 'thrustMagAll', 'thrustCosTh', 'thrustCosThAll', 'sphericityAll', \
          'BCosSphr', 'BCosThetaT', 'BCosThrust', 'BLegendreP2', 'BR2ROE', 'BSphrROE', \
          'BThrustROE']
'''
#subset = ['BSphr', 'BpostFitDeltaE', 'BpostFitMes', 'BThrust', 'BCosThetaS']
#subset = ['Lambda0_unc_Mass']
subset = ['Lambda0_unc_Mass', 'BpostFitMes', 'BpostFitDeltaE']

all_hists= bat.create_empty_histograms(hist_defs)

bkg_modes =['998', '1005', '1235', '1237', '3981']
if BNC_bool:
    # No events survived 3981
    bkg_modes =['998', '1005', '1235', '1237']
    

sig_modes = ['-999']

spmodes = bkg_modes + sig_modes

#cut_idx = -1
#cut_name = str(cut_idx)
#cut = dcuts[cut_idx]['event']
#cut_sig = dcuts_sig[cut_idx]['event']

#df = bat.fill_histograms_v3(data[cut], subset=subset, empty_hists=all_hists, spmodes=bkg_modes + sig_modes, weights=weights, cuts=cut_name)
df_cutflow_sp = bat.fill_histograms_v3(data, subset=subset, empty_hists=all_hists, spmodes=spmodes, weights=weights, cuts=dcuts);
df_cutflow_sp = bat.fill_histograms_v3(data_collision, subset=subset, empty_hists=all_hists, spmodes=['0'], weights=weights, cuts=dcuts_coll);


#df = bat.fill_histograms_v3(data, subset=['BpostFitDeltaE'], empty_hists=all_hists, spmodes=bkg_modes + sig_modes, weights=weights)


In [ ]:
for key,vals in dcuts.items():
    print(f"{key:4d}   {vals['name']}")

In [ ]:
dcuts.keys()

In [ ]:
all_vars = list(all_hists.keys())

print(all_vars)
print(len(all_vars))
print(len(subset))

#bat.plot_histograms(all_hists, vars=subset, bkg_spmodes=bkg_modes, sig_spmodes=sig_spmodes, datamodes=['0'], cut=cut_name, overlay_data=False, only_stacked=True, save= True, fixed_grid=(4,5));

In [ ]:
'''
cut_idx = 1
cut_name = str(cut_idx)
cut = dcuts[cut_idx]['event']
cut_sig = dcuts_sig[cut_idx]['event']
'''

In [ ]:
#all_hists['Lambda0_unc_Mass'][:,['3981'],'1']

In [ ]:
#### BACKGROUNDS LAMBDA MASS 
# For BAD

for cut_name in ['1', '2', '3', '4', '6', '-1', '-2']:
    #cut_name = '-1'
    
    bat.plot_histograms(all_hists, vars=subset, bkg_spmodes=bkg_modes, sig_spmodes=sig_spmodes, datamodes=['0'], cut=cut_name, overlay_data=True, only_stacked=True, save= True, fixed_grid=(1,3), extra_tag=BNC_tag);


In [ ]:
#all_hists['Lambda0_unc_Mass'][:,'0','3'].project('var').plot(histtype="step", color='black', label= "signal")


In [ ]:
'''
df = bat.fill_histograms_v3(signal[cut_sig], subset=subset, empty_hists=all_hists, spmodes=sig_modes+bkg_spmodes, weights=weights, cutname=cut_name)

bat.plot_histograms(all_hists, vars=subset, bkg_spmodes=bkg_spmodes, sig_spmodes=sig_spmodes, datamodes=['0'], cut=cut_name, overlay_data=False, only_stacked=True, save= True, fixed_grid=(4,5), extra_tag=BNC_tag);
#plt.ylim(0,2500)
'''

In [ ]:
#plt.figure()
#plt.hist(ak.flatten(signal["Lambda0_unc_Mass"]), bins= 100)
#plt.xlabel("Mass $[GeV/c^2]$")
#plt.ylabel("Frequency")
#plt.title("Signal- $\Lambda^0$ Mass")

In [ ]:
plt.hist(ak.flatten(collision["Lambda0_unc_Mass"]), bins= 100)
plt.xlabel("Mass $[GeV/c^2]$")
plt.ylabel("Frequency")
plt.title("Collision- $\Lambda^0$ Mass")

In [ ]:
#### MES DE PLOT 
region_definitions

In [ ]:
def plot_mes_vs_DeltaE(mes, DeltaE, draw_signal_region=False, draw_sidebands=False, draw_inference_bins=False, tag=None, region_definitions=None, bins=100, zoom=False):

    meslo = region_definitions['fitting MES'][0]
    meshi = region_definitions['fitting MES'][1]
    DeltaElo = -0.5
    DeltaEhi =  0.5

    sigmeslo = region_definitions['signal MES'][0]
    sigmeshi = region_definitions['signal MES'][1]
    sigDeltaElo = region_definitions['signal DeltaE'][0]
    sigDeltaEhi = region_definitions['signal DeltaE'][1]

    sbDE1lo = region_definitions['sideband 1 DeltaE'][0]
    sbDE1hi = region_definitions['sideband 1 DeltaE'][1]
    sbDE2lo = region_definitions['sideband 2 DeltaE'][0]
    sbDE2hi = region_definitions['sideband 2 DeltaE'][1]

    sbmeslo = region_definitions['sideband MES'][0]
    sbmeshi = region_definitions['sideband MES'][1]

    if zoom==True and region_definitions is not None:
        DeltaElo = region_definitions['fitting DeltaE'][0]
        DeltaEhi = region_definitions['fitting DeltaE'][1]

    h= Hist(
        hist.axis.Regular(bins,meslo,meshi,name= "sig_BPFM", label= "M$_{ES}$ [GeV/c$^2$]", flow= True),
        hist.axis.Regular(bins,DeltaElo,DeltaEhi,name= "bkg_BPFMDE", label= "$\Delta$E [GeV]", flow= True),
    )

    # normal fill
    h.fill(mes, DeltaE)
    print(h)

In [ ]:
###################################
# SP
###################################

#splist= np.unique(sp.to_list())
#print(splist)

dcuts= bat.get_final_masks(data, region_definitions= region_definitions)

print([dcuts.keys()])
print()

for key in dcuts.keys():
    print(f'{key:3d} {dcuts[key]["name"]}')


###################################
# Collision
###################################

dcuts_coll= bat.get_final_masks(data_collision, region_definitions= region_definitions)




In [ ]:
# SP

bkg_modes =['998', '1005', '1235', '1237', '3981', '3429']
sig_modes = ['-999']

spmodes = bkg_modes + sig_modes

for spmode in spmodes:

    plt.figure()
    
    mask = (data['spmode'] == spmode) & (dcuts[1]['event'])# & (dcuts[2]['event'])
    
    Mes= ak.flatten(data[mask]["BpostFitMes"])
    DeltaE= ak.flatten(data[mask]["BpostFitDeltaE"])
    tag = f'sp{spmode}_all_events_cut1{BNC_tag}'
    bat.plot_mes_vs_DeltaE(Mes, DeltaE, region_definitions= region_definitions,draw_signal_region=True, draw_fit_region=True, tag=tag)


In [ ]:
# Collision data

for spmode in ['0']:

    plt.figure()
    
    mask = (data_collision['spmode'] == spmode) & (dcuts_coll[1]['event'])# & (dcuts_coll[2]['event'])
    
    Mes= ak.flatten(data_collision[mask]["BpostFitMes"])
    DeltaE= ak.flatten(data_collision[mask]["BpostFitDeltaE"])
    tag = f'collision_data_all_events_cut1{BNC_tag}'
    bat.plot_mes_vs_DeltaE(Mes, DeltaE, region_definitions= region_definitions,draw_signal_region=True, draw_fit_region=True, tag=tag)


In [ ]:
# Background

#Mes= ak.flatten(data["BpostFitMes"])
#DeltaE= ak.flatten(data["BpostFitDeltaE"])
#bat.plot_mes_vs_DeltaE(Mes, DeltaE, region_definitions= region_definitions)


In [ ]:
# Collision

#Mes= ak.flatten(collision["BpostFitMes"])
#DeltaE= ak.flatten(collision["BpostFitDeltaE"])
#plt.figure(figsize=(4,4))
#bat.plot_mes_vs_DeltaE(Mes, DeltaE, region_definitions= region_definitions)



In [ ]:
def plotem(mes, DeltaE, draw_signal_region=False, draw_sidebands=False, draw_inference_bins=False, tag=None, region_definitions=None, bins=100, zoom=False):

    meslo = region_definitions['fitting MES'][0]
    meshi = region_definitions['fitting MES'][1]
    DeltaElo = -0.5
    DeltaEhi =  0.5

    sigmeslo = region_definitions['signal MES'][0]
    sigmeshi = region_definitions['signal MES'][1]
    sigDeltaElo = region_definitions['signal DeltaE'][0]
    sigDeltaEhi = region_definitions['signal DeltaE'][1]

    sbDE1lo = region_definitions['sideband 1 DeltaE'][0]
    sbDE1hi = region_definitions['sideband 1 DeltaE'][1]
    sbDE2lo = region_definitions['sideband 2 DeltaE'][0]
    sbDE2hi = region_definitions['sideband 2 DeltaE'][1]

    sbmeslo = region_definitions['sideband MES'][0]
    sbmeshi = region_definitions['sideband MES'][1]

    if zoom==True and region_definitions is not None:
        DeltaElo = region_definitions['fitting DeltaE'][0]
        DeltaEhi = region_definitions['fitting DeltaE'][1]

    h= Hist(
        hist.axis.Regular(bins,meslo,meshi,name= "sig_BPFM", label= "M$_{ES}$ [GeV/c$^2$]", flow= True),
        hist.axis.Regular(bins,DeltaElo,DeltaEhi,name= "bkg_BPFMDE", label= "$\Delta$E [GeV]", flow= True),
    )

    # normal fill
    h.fill(mes, DeltaE)
    return h

In [ ]:
Mes= ak.flatten(signal["BpostFitMes"])
DeltaE= ak.flatten(signal["BpostFitDeltaE"])
signal_unfilled= plotem(Mes, DeltaE, region_definitions= region_definitions)
signal_filled= signal_unfilled.plot2d_full(
            #main_cmap="coolwarm",
        main_cmap="plasma",
        top_ls="--",
        top_color="orange",
        top_lw=2,
        side_ls=":",
        side_lw=2,
        side_color="steelblue",
    )



In [ ]:
### Put the histograms into subplots based on them BEFORE they're filled using ChatGPT's advice 

'''
import matplotlib.pyplot as plt
import numpy as np
from hist import Hist

# Create some data
data1 = np.random.randn(1000)
data2 = np.random.randn(1000)

# Create a Hist object
h = Hist.new.Reg(50, -3, 3).Double()
h.fill(data1)

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Plot a regular matplotlib histogram
axes[0].hist(data2, bins=30, alpha=0.7, color='blue')
axes[0].set_title("Matplotlib Histogram")

# Plot the hist.Hist object explicitly on the second subplot
h.plot(ax=axes[1])  # <- This ensures it goes into the right subplot
axes[1].set_title("Hist Histogram")

plt.tight_layout()
plt.show()
'''

In [ ]:
import hist
from hist import Hist

In [ ]:
# Bellis edits

# Need to get the original duplicates mask for any other cuts we might generate outside the function
# SP - bkg
print("Background")
dcuts_bkg = bat.get_final_masks(data, region_definitions=region_definitions)
for key in dcuts_bkg.keys():
    print(f'{key:3d} {dcuts_bkg[key]["name"]}')

# SP - sig
print("Signal")
dcuts_sig = bat.get_final_masks(signal, region_definitions=region_definitions)
for key in dcuts_sig.keys():
    print(f'{key:3d} {dcuts_sig[key]["name"]}')

# Collision
print("Collision")
dcuts_coll = bat.get_final_masks(collision, region_definitions=region_definitions)
for key in dcuts_coll.keys():
    print(f'{key:3d} {dcuts_coll[key]["name"]}')

In [ ]:
# Collision

cut_idx = -1
cut_name = str(cut_idx)
cut_coll = dcuts_coll[cut_idx]['event']
cut_sig = dcuts_sig[cut_idx]['event']
cut_bkg = dcuts_bkg[cut_idx]['event']


coll_Mes= ak.flatten(collision[cut_coll]["BpostFitMes"])
coll_DeltaE= ak.flatten(collision[cut_coll]["BpostFitDeltaE"])

# Background

bkg_Mes= ak.flatten(data[cut_bkg]["BpostFitMes"])
bkg_DeltaE= ak.flatten(data[cut_bkg]["BpostFitDeltaE"])

# Signal

sig_Mes= ak.flatten(signal[cut_sig]["BpostFitMes"])
sig_DeltaE= ak.flatten(signal[cut_sig]["BpostFitDeltaE"])


In [ ]:
coll_2dhist = Hist.new.Reg(100, region_definitions['fitting MES'][0], region_definitions['fitting MES'][1]).Reg(100, -0.5, 0.5).Double()
coll_2dhist.fill(coll_Mes, coll_DeltaE)

bkg_2dhist = Hist.new.Reg(100, region_definitions['fitting MES'][0], region_definitions['fitting MES'][1]).Reg(100, -0.5, 0.5).Double()
bkg_2dhist.fill(bkg_Mes, bkg_DeltaE)

sig_2dhist = Hist.new.Reg(100, region_definitions['fitting MES'][0], region_definitions['fitting MES'][1]).Reg(100, -0.5, 0.5).Double()
sig_2dhist.fill(sig_Mes, sig_DeltaE)

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(10, 10))

bkg_2dhist.plot(ax=axes[0, 0])  
axes[0,0].set_title("Background")
axes[0,0].set_xlabel("$M_{ES}$ $[GeV/c^2]$")
axes[0,0].set_ylabel("$\Delta$E")

sig_2dhist.plot(ax=axes[1, 0])  
axes[1,0].set_title("Signal")
axes[1,0].set_xlabel("$M_{ES}$ $[GeV/c^2]$")
axes[1,0].set_ylabel("$\Delta$E")

coll_2dhist.plot(ax=axes[2, 0])  
axes[2,0].set_title("Collision")
axes[2,0].set_xlabel("$M_{ES}$ $[GeV/c^2]$")
axes[2,0].set_ylabel("$\Delta$E")


axes[0,1].hist(ak.flatten(data[cut_bkg]["Lambda0_unc_Mass"]), range=(1.105, 1.125), bins= 100)
axes[0,1].set_xlabel("Mass $[GeV/c^2]$")
axes[0,1].set_ylabel("Frequency")
axes[0,1].set_title("Background- $\Lambda^0$ Mass")

axes[1,1].hist(ak.flatten(signal[cut_sig]["Lambda0_unc_Mass"]), range=(1.105, 1.125), bins= 100)
axes[1,1].set_xlabel("Mass $[GeV/c^2]$")
axes[1,1].set_ylabel("Frequency")
axes[1,1].set_title("Signal- $\Lambda^0$ Mass")

axes[2,1].hist(ak.flatten(collision[cut_coll]["Lambda0_unc_Mass"]), range=(1.105, 1.125), bins= 100)
axes[2,1].set_xlabel("Mass $[GeV/c^2]$")
axes[2,1].set_ylabel("Frequency")
axes[2,1].set_title("Collision- $\Lambda^0$ Mass")

plt.tight_layout()


plt.savefig(f'grid6_cut_{cut_name}.png')



# Cutflow tables

In [ ]:
%load_ext autoreload
%autoreload 2

from analysis_variables import *


# Need to get the original duplicates mask for any other cuts we might generate outside the function
dcuts = bat.get_final_masks(data, region_definitions=region_definitions)

print([dcuts.keys()])
print()

for key in dcuts.keys():
    print(f'{key:3d} {dcuts[key]["name"]}')

print()

#'''
# Need to get the original duplicates mask for any other cuts we might generate outside the function
dcuts_coll = bat.get_final_masks(data_collision, region_definitions=region_definitions)

print([dcuts_coll.keys()])
print()

for key in dcuts_coll.keys():
    print(f'{key:3d} {dcuts_coll[key]["name"]}')
#'''

In [ ]:
df1005_cutflow= bat.get_numbers_for_cut_flow(data, region_definitions=region_definitions,tag="DEFAULT", \
                                             spmodes=["1005"], verbose=False, dcuts=dcuts)
df1005_cutflow

In [ ]:
dfsig_cutflow= bat.get_numbers_for_cut_flow(data, region_definitions=region_definitions,tag="DEFAULT", \
                                            spmodes=["-999"], verbose=False, dcuts=dcuts)
dfsig_cutflow

In [ ]:
df1005_cutflow= df1005_cutflow.rename(columns={"name": "name_1005", "pct": "pct_1005","tag":"tag_1005","nevents":"nevents_1005"})

In [ ]:
dfsig_cutflow= dfsig_cutflow.rename(columns={"name": "name_sig", "pct": "pct_sig","tag":"tag_sig","nevents":"nevents_sig"})

In [ ]:
df1005_cutflow= df1005_cutflow.drop(["tag_1005","spmode"],axis=1).rename(columns= {"nevents":"nevents_1005","pct":"pct_1005"})

In [ ]:
cutflow_df_compiled= df1005_cutflow.merge(dfsig_cutflow, on= "cut").drop(["name_sig","spmode","tag_sig","name_1005"],axis=1).rename(columns= {"name_1005":"cut_name"})
cutflow_df_compiled

In [ ]:
df998_cutflow= bat.get_numbers_for_cut_flow(data, region_definitions=region_definitions,tag="DEFAULT", \
                                            spmodes=["998"], verbose=False, dcuts=dcuts)

In [ ]:
df998_cutflow= df998_cutflow.drop(["spmode","tag","name"], axis= 1).rename(columns= {"nevents":"nevents_998", "pct":"pct_998"})

In [ ]:
cutflow_df_compiled= cutflow_df_compiled.merge(df998_cutflow, on= "cut")

In [ ]:
cutflow_df_compiled= cutflow_df_compiled.rename(columns= {"nevents_1005":r"$\#$ $c\bar{c}$","pct_1005":r"$\%$ $c\bar{c}$","nevents_sig":"$\#$ signal","pct_sig":"$\%$ signal","nevents_998":r"$\#$ $u\bar{u},d\bar{d},s\bar{s}$","pct_998":r"$\% u\bar{u},d\bar{d},s\bar{s}$"})

In [ ]:
collision_cutflow= bat.get_numbers_for_cut_flow(data_collision, region_definitions=region_definitions,tag="DEFAULT", \
                                                spmodes=["0"], verbose=False, dcuts=dcuts_coll)
collision_cutflow= collision_cutflow.drop(["name","spmode","tag"], axis= 1).rename(columns= {"nevents":r"$\#$ data","pct":r"$\%$ data"})

In [ ]:
cutflow_df_compiled= cutflow_df_compiled.merge(collision_cutflow, on= "cut")


In [ ]:
cutflow_df_compiled= cutflow_df_compiled[[r"cut", r"$\#$ signal", r"$\%$ signal", r"$\#$ $u\bar{u},d\bar{d},s\bar{s}$", r"$\% u\bar{u},d\bar{d},s\bar{s}$", r"$\#$ $c\bar{c}$", r"$\%$ $c\bar{c}$" ,  r"$\#$ data"]]

cutflow_df_compiled

In [ ]:
cutflow_df_compiled["cut"][0]= str("None")
cutflow_df_compiled["cut"][1]= str("Duplicates")
cutflow_df_compiled["cut"][2]= str("Fit Area")
cutflow_df_compiled["cut"][3]= str("Lambda")
cutflow_df_compiled["cut"][4]= str("PID")
cutflow_df_compiled["cut"][5]= str("Antibaryon")
cutflow_df_compiled["cut"][6]= str("All cuts")


In [ ]:
cutflow_df_compiled

In [ ]:
output = cutflow_df_compiled.to_latex(index=False,
              float_format="{:.1f}".format,
)
#full_table = "\\begin{table}\n"  
#full_table += "\\caption{Cutflow}\n" 
#full_table += output 
#full_table += "\\end{table}" 

# Just the tabular
full_table = output

print(full_table)

outfile = open('tables/table_cutflow_stats.tex', 'w')
outfile.write(full_table)
outfile.close()